In [5]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
cd gdrive/My\ Drive/DMIS/Re-Implementation/Transformer_Re-Implementation

/content/gdrive/My Drive/DMIS/Re-Implementation/Transformer_Re-Implementation


In [2]:
!nvidia-smi

Thu Aug  8 05:22:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## ================== drive mounting ======================
## Train

In [0]:
import model as trf
from process import get_data
from train import *
from config import *

In [0]:
import torch
import torch.nn as nn
from torchtext import data, datasets

In [1]:
data_path = "dataset/"
file_name = "train."
save_path = "model_save/transformer_epoch"

In [0]:
SRC, TGT, trn = get_data(data_path + file_name)

In [0]:
pad_idx = TGT.vocab.stoi["<pad>"]

In [0]:
model = trf.Transformer(len(SRC.vocab), len(TGT.vocab), N = 6)
model.cuda()
criterion = nn.NLLLoss(ignore_index = 1)
criterion.cuda()
model_opt = torch.optim.Adam(model.parameters(), lr = .1, betas=(.9, .98), eps = 1e-9)

In [0]:
train_iter = data.Iterator(trn, batch_size = BATCH_SIZE,
                          device = DEVICE_SET,
                          repeat = False)

In [0]:
train_model(epochs, model, criterion, model_opt, train_iter, save_path, print_every)

Iteration Step: 0 Loss: 0.113218 per Sec: 4.277717
Iteration Step: 100 Loss: 17.789034 per Sec: 375.880995
Iteration Step: 200 Loss: 9.439085 per Sec: 400.231872
Iteration Step: 300 Loss: 10.091720 per Sec: 420.132053
Iteration Step: 400 Loss: 10.584603 per Sec: 455.191933
Iteration Step: 500 Loss: 10.080555 per Sec: 461.691407
Iteration Step: 600 Loss: 10.324879 per Sec: 450.547680
Iteration Step: 700 Loss: 13.177295 per Sec: 485.454046
Iteration Step: 800 Loss: 10.903852 per Sec: 542.740345
Iteration Step: 900 Loss: 10.482918 per Sec: 564.208768
Iteration Step: 1000 Loss: 15.744347 per Sec: 539.490917
Iteration Step: 1100 Loss: 11.308460 per Sec: 567.702864
Iteration Step: 1200 Loss: 11.630238 per Sec: 582.721534
Iteration Step: 1300 Loss: 12.407413 per Sec: 589.636211
Iteration Step: 1400 Loss: 18.401762 per Sec: 610.731749
Iteration Step: 1500 Loss: 11.632895 per Sec: 587.646272
Iteration Step: 1600 Loss: 19.478905 per Sec: 608.171128
Iteration Step: 1700 Loss: 11.323104 per Sec: 5

KeyboardInterrupt: ignored